In [4]:
import pandas as pd

In [5]:
path = 'support_data/comics.json'

In [7]:
comics = pd.read_json(path, lines=True)

In [10]:
comic_ids = comics['comic_id'].tolist()

In [11]:
comic_titles = comics['comic_title'].tolist()

In [13]:
comic_ids[:4]

[1, 2, 3, 4]

In [14]:
comic_titles[:10]

['0Secret Wars (Marvel)',
 '100 Bullets Brother Lono (DC)',
 '100 Penny Press Locke & Key (IDW)',
 '100 Penny Press Star Trek (IDW)',
 '100 Penny Press Thunder Agent (IDW)',
 '100 Penny Press Tmnt (IDW)',
 '100th Anniversary Special (Marvel)',
 '12 Reasons To Die (Other)',
 '13 Coins (Other)',
 '13th Artifact One Sho (Topcow)']

In [2]:
!pwd

/Users/werlindo/Dropbox/flatiron/capstone/comics_rx/comrx/dev


In [6]:
import pandas as pd
import requests
import random
import time
import os

# Data storage
from sqlalchemy import create_engine # SQL helper
import keys     # Custom keys lib
import psycopg2 as psql #PostgreSQL DBs

In [3]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
                           'aws_ps_flatiron.json')
secret_path_aws

'/Users/werlindo/.secret/aws_ps_flatiron.json'

In [7]:
aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [8]:
# Instantiate cursor
cur = conn.cursor()

In [24]:
#  Count records.
query = """
    SELECT ct.*, c.comic_id from comic_trans ct
    inner join comics c on ct.comic_title = c.comic_title;
"""

In [25]:
# Execute the query
cur.execute(query)

In [26]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

temp_df.head(3)

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,comic_id
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),2291
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),2445
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),5178


In [29]:
qtys = (temp_df.groupby(['comic_id','comic_title'], as_index=False)
        .qty_sold.sum().sort_values(by=['qty_sold'], ascending=False))

In [30]:
top_100 = qtys.head(100)

In [55]:
top_100 = (top_100.sort_values(by=['comic_title'], ascending=True).
                    loc[:,['comic_id', 'comic_title']])

### Export back out to csv

In [56]:
top_100.to_csv('./webapp/templates/dev_files/top_100_comics.csv', 
               index=False)

### Make lists

In [42]:
coms = pd.read_csv('./webapp/templates/dev_files/top_100_comics.csv')

In [43]:
coms.head()

,comic_id,comic_title
0,81,Action Comics (DC)
1,198,All New X-Men (Marvel)
2,224,Amazing Spider-Man (Marvel)
3,278,Animal Man (DC)
4,319,Aquaman (DC)


In [48]:
ids = coms['comic_id'].tolist()

In [49]:
titles = coms['comic_title'].tolist()

In [51]:
ids[:10]

[81, 198, 224, 278, 319, 388, 433, 455, 552, 610]

In [52]:
titles[:10]

['Action Comics (DC)',
 'All New X-Men (Marvel)',
 'Amazing Spider-Man (Marvel)',
 'Animal Man (DC)',
 'Aquaman (DC)',
 'Astonishing X-Men (Marvel)',
 'Avengers (2010) (Marvel)',
 'Avengers (Marvel)',
 'Batgirl (DC)',
 'Batman (DC)']